In [ ]:
# Building and Trainig the Model

import keras

encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

from keras.optimizers import Adam

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=["accuracy"])

model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
           batch_size=batch_size,epochs=epochs,validation_split=0.2,)

In [18]:
tokens_eng = [' ', '!', '$', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 
              'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
              'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 
              'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

tokens_spn = ['N', '\t', 'B', 'G', '"', 'I', 'f', 'i', 'ó', '.', ',', 'Á', 'Ó', 'E', 'í', '\n', 'y', 'C', 'Y', 'z',
              'p', 'q', 'A', 'u', 'l', 'ñ', '¡', 'U', 'r', 'k', 't', '?', 'h', 'n', '7', 'á', 'S', '2', ':', 'É',
              'ü', 'v', 'V', ' ', '8', '0', 'W', 'o', 'T','¿', 'e', 'ú', 'g', '»', 'd', '!', 'P', 'H', 's', 'L',
              'O', '«', 'b', 'c', '5', '4', 'm', "'", 'R', '3', 'x', 'w', 'j', 'Ú', 'J', '6', 'é', 'D', 'M',
              'F', '1', 'a', '-', 'Q']

In [26]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, 69)]   0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, None, 84)]   0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 333824      input_4[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  349184      input_5[0][0]                    
                                                                 lstm_3[0][1]               

In [62]:
char2_int_eng = dict(((x,y) for y,x in enumerate(tokens_eng,0)))
int2_char_eng = dict(((y,x) for y,x in enumerate(tokens_eng,0)))

In [63]:
char2_int_spn = dict(((x,y) for y,x in enumerate(tokens_spn,0)))
int2_char_spn = dict(((y,x) for y,x in enumerate(tokens_spn,0)))

In [64]:
# Let us load the saved model above the same

from tensorflow import keras
model = keras.models.load_model('E2S')

In [65]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, 69)]   0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, None, 84)]   0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 333824      input_4[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  349184      input_5[0][0]                    
                                                                 lstm_3[0][1]               

In [66]:
# Decode the Sentence
encoder_inputs = model.input[0]  # input_1

encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1

encoder_states = [state_h_enc, state_c_enc] # encoder hidden states

# Encoder model required only Encoder input and ender hidden states
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [67]:
# Step 1: Predict the hidden states from encoded model for use input sequence

decoder_inputs = model.input[1]

decoder_state_input_h = keras.Input(shape=(256,))
decoder_state_input_c = keras.Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]


decoder_lstm = model.layers[3]
decoder_outputs,decoder_state_h,decoder_state_h = decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
decoder_output_states = [decoder_state_h,decoder_state_h]

decoder_input_states = [decoder_state_input_h,decoder_state_input_c]

# Decoder Model requires decoder input and decoder hidden states as first parameter and 
# decoder output and decoder output hidden states as second parameters
decoder_model = keras.Model([decoder_inputs] + decoder_input_states,[decoder_outputs] + decoder_output_states)


In [138]:
import numpy as np

def generateEncodinForPrediction(text):
    
    # create array of Zero with size of Max_length_seq(time_step) and Vocab_size
    decode_input_seq = np.zeros((16,69))
    
    for i,char in enumerate(text):
        
        # Update the char values present
        decode_input_seq[i,char2_int_eng[char]] = 1
        
        
        
    return np.asarray([decode_input_seq]) # return the 3d converted array

In [140]:
input_seq = generateEncodinForPrediction("Go.")

input_seq[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [132]:
states_value = encoder_model.predict(input_seq) # combine 'states_value' 

In [133]:
target_seq = np.zeros((1, 1, len(char2_int_spn)))
target_seq[0, 0, char2_int_spn["\t"]] = 1.0

In [134]:
output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

In [135]:
sampled_token_index = np.argmax(output_tokens[0, -1, :])